In [2]:
# Saber en cuanto podría vender su coche

- Leyendo el dataset

In [3]:
import pandas as pd

In [4]:
cars = pd.read_csv("cars.csv")

In [5]:
cars.head()

,maker,model,year,transmission,mileage,fuelType,tax,mpg,engineSize,price
0,cclass,C Class,2020,Automatic,1200,Diesel,NaN,NaN,2.0,30495
1,cclass,C Class,2020,Automatic,1000,Petrol,NaN,NaN,1.5,29989
2,cclass,C Class,2020,Automatic,500,Diesel,NaN,NaN,2.0,37899
3,cclass,C Class,2019,Automatic,5000,Diesel,NaN,NaN,2.0,30399
4,cclass,C Class,2019,Automatic,4500,Diesel,NaN,NaN,2.0,29899


- Análisis exploratorio de datos

In [6]:
from pandas_profiling import ProfileReport

In [7]:
# Cremos un objeto de la clase ProfileReport y le pasamos el dataframe, con titulo y usamos explorative como verdadero (para generar un reporte más extenso).
# 2.- Después como a ese objeto vamos a llamar un método llamado to_file y le pasamos como nombre "reporte.html"

In [8]:
profile = ProfileReport(cars, title="Raw Car Dataset Analysis", explorative=True)  # 1
profile.to_file("reporte.html") # 2

Export report to file: 100%|██████████| 1/1 [00:00<00:00,  3.09it/s]


- Elimina los duplicados

In [9]:
# 1.- Vamos a usar un método de pands para eliminar valores duplicados.

In [10]:
len(cars)

108540

In [11]:
cars = cars.drop_duplicates(keep="first") # 1
len(cars)

106267

- Dividimos nuestro Dataset

In [12]:
# 1.- de model_selection voy a importar una función que se llama train_test_split
# 2.-


In [13]:
from sklearn.model_selection import train_test_split  # 1
import numpy as np

In [14]:
rest, test = train_test_split(cars, test_size=0.2, shuffle=True) # 20% of 100 = 20
# Me permite especificar mi dataset, el tamaño que quiero (en %) y esepcificar que los barajeé


train, val = train_test_split(rest, test_size=0.25, shuffle=True) # 25% of 80 = 20
# Tomo el conjunto restante por eso "rest"

distributions = np.array([len(train), len(val), len(test)])

print(distributions)
print(distributions / len(cars))

[63759 21254 21254]
[0.59998871 0.20000565 0.20000565]


- Feature Engineering

In [15]:
# Las variables categoricas al NO ser números, debemos pasarlas a números para poder trabajar con ellas
# Pandas tiene un método para convertir en dummies
#


In [16]:
from sklearn.preprocessing import OneHotEncoder
maker_encoder = OneHotEncoder() # Creamos objeto que se llama OneHotEncoder


In [17]:
# 1.- Utilizo ese objeto y le voy a decir: "Oye, prepara con esos datos(datos de entrenamiento)"
# 2.- Puedo usar el metodo tranform y me dará una matriz
# 2.- Para visualizar la matriz (NO ES NECESARIO USAR) se usa ".todense()"

In [18]:
maker_encoder.fit(train[["maker"]]) # 1

OneHotEncoder()

In [19]:
mkr = maker_encoder.transform(train[["maker"]]).todense()  # 2
mkr

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 1., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
# pd.get_dummies(train[["maker"]]) 
# NO RECOMENDABLE PARA MODELO DE ML
# get_dummies NO guarda el estado de mis vairbales

In [21]:
maker_encoder.categories_ # con este metodo

[array(['audi', 'bmw', 'cclass', 'focus', 'ford', 'hyundi', 'merc',
        'skoda', 'toyota', 'vauxhall', 'vw'], dtype=object)]

In [22]:
# Creo un DataFrame a partir de mkr


In [23]:
df = pd.DataFrame(mkr, columns=maker_encoder.categories_, index=train[["maker"]].index)
df["actual"] = train[["maker"]]
df.sample(5)

,audi,bmw,cclass,focus,ford,hyundi,merc,skoda,toyota,vauxhall,vw,actual
25607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,toyota
39404,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ford
3026,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,cclass
27846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,skoda
202,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,cclass


- Feature scaling (comprimir)

In [24]:
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, MaxAbsScaler
scaler = MaxAbsScaler()

In [25]:
# Tomo el objeto y lo preparo
# ¿Cuando se usa cada escalador?

In [26]:
scaler.fit(train[["mileage"]])

MaxAbsScaler()

In [27]:
scaled = scaler.transform(train[["mileage"]])
scaled

array([[0.16553   ],
       [0.03703   ],
       [0.08333333],
       ...,
       [0.01333333],
       [0.00972333],
       [0.06899333]])

In [28]:
# Intentamos hacer que los números sean mas pequeños, pero con escala, para que el modelo pueda entender mejor.

In [29]:
values = pd.DataFrame({"mileage": train["mileage"].values, "scaled": scaled.squeeze() })
values.sample(5)

,mileage,scaled
49171,20025,0.066750
59052,6875,0.022917
26565,20567,0.068557
60302,16039,0.053463
6032,93839,0.312797


- Artefactos

In [30]:
# Sirven para transformar una de nuestras observaciones del munddo real, como el diálogo emitido por una persona o un automóvil, a un grupo de números.
# Cosas como el OneHotEncoder, CountVectorizer y MaxAbsScaler forman parte de este conjunto de herramientas que, una vez preparadas con fit, debemos preservar para poder re-usarlas en producción.
# Estas herramientas son conocidas como artefactos.

In [31]:
# Una forma de guardar los artefactos es con el serializador 
# Creo archivo scaler en formato binario "wb"

In [32]:
import pickle

with open("scaler.pickle", "wb") as wb:
    pickle.dump(scaler, wb)


- Pipelines

In [33]:
# Permite agrupar, guardar y ponerlo en producción de manera sencilla

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn import set_config

In [35]:
# One-Hot encode maker, transmission y fuelType
one_hot_encode = ColumnTransformer([ # con esto decimos que columnas vamos a transformar 
    (
        'one_hot_encode', # Nombre de la transformación
        OneHotEncoder(sparse=False), # Transformación a aplicar
        ["maker", "transmission", "fuelType"] # Columnas involucradas
    )
])

In [36]:
# sparse hace que la matriz se gaurde en poca memoria

In [37]:
one_hot_encode.fit(train)

ColumnTransformer(transformers=[('one_hot_encode', OneHotEncoder(sparse=False),
                                 ['maker', 'transmission', 'fuelType'])])

In [38]:
variable = one_hot_encode.transform(train)
variable.shape # son 20 columnas porque suma las columnas
# 11 de maker + 4 de tranmission + 5 de fuelType

(63759, 20)

In [39]:
variable

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [40]:
# Robust encode mileage
robust_encoding = ColumnTransformer([
    ('robust_encoding', RobustScaler(), ["mileage"])
])

In [41]:
# Impute and standard scale mpg and tax
impute_and_scale = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler())
])

standard_scaling = ColumnTransformer([
    ('standard_scaling', impute_and_scale, ["mpg", "tax"])
])

In [42]:
# Aquí dice: Las vairbales que te voy a dar pasalas igual, no les hagas nada
#

In [43]:
# Just pass year and engineSize 
passthrough = ColumnTransformer([('passthrough', 'passthrough', ['year', "engineSize"])])


In [44]:
# Ensambla todo el pipeline
pipe = Pipeline([
    (
        'features',
        FeatureUnion([ # Toma todos los resultadis de columntranformer y los va a unir
            ('one_hot_encode', one_hot_encode),
            ('robust_encoding', robust_encoding),
            ('just_passs', passthrough),
            ('scale_and_impute', standard_scaling)
        ])
    )
])

In [45]:
from sklearn import set_config

set_config(display="diagram")
pipe

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('one_hot_encode',
                                                 ColumnTransformer(transformers=[('one_hot_encode',
                                                                                  OneHotEncoder(sparse=False),
                                                                                  ['maker',
                                                                                   'transmission',
                                                                                   'fuelType'])])),
                                                ('robust_encoding',
                                                 ColumnTransformer(transformers=[('robust_encoding',
                                                                                  RobustScaler(),
                                                                                  ['mileage'])])),
                                                ('just_passs',
                                                 ColumnTransformer(transformers=[('passthrough',
                                                                                  'passthrough',
                                                                                  ['year',
                                                                                   'engineSize'])])),
                                                ('scale_and_impute',
                                                 ColumnTransformer(transformers=[('standard_scaling',
                                                                                  Pipeline(steps=[('impute',
                                                                                                   SimpleImputer()),
                                                                                                  ('scale',
                                                                                                   MinMaxScaler())]),
                                                                                  ['mpg',
                                                                                   'tax'])]))]))])

In [46]:
pipe.fit(train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('one_hot_encode',
                                                 ColumnTransformer(transformers=[('one_hot_encode',
                                                                                  OneHotEncoder(sparse=False),
                                                                                  ['maker',
                                                                                   'transmission',
                                                                                   'fuelType'])])),
                                                ('robust_encoding',
                                                 ColumnTransformer(transformers=[('robust_encoding',
                                                                                  RobustScaler(),
                                                                                  ['mileage'])])),
                                                ('just_passs',
                                                 ColumnTransformer(transformers=[('passthrough',
                                                                                  'passthrough',
                                                                                  ['year',
                                                                                   'engineSize'])])),
                                                ('scale_and_impute',
                                                 ColumnTransformer(transformers=[('standard_scaling',
                                                                                  Pipeline(steps=[('impute',
                                                                                                   SimpleImputer()),
                                                                                                  ('scale',
                                                                                                   MinMaxScaler())]),
                                                                                  ['mpg',
                                                                                   'tax'])]))]))])

In [47]:
train_x = pipe.transform(train)
train_x.shape # el 25 es por la suma de todas las columnas

(63759, 25)

- Modelado

In [48]:
from sklearn.linear_model import LinearRegression

In [49]:
lr = LinearRegression()

In [ ]:
# Creamos otro pipeline que incluya nuestro modelo de regresión 

In [50]:
predicting_pipeline = Pipeline([
    ('feature', pipe),
    ('estimator', lr)
])

In [51]:
predicting_pipeline.fit(train, train['price'])

Pipeline(steps=[('feature',
                 Pipeline(steps=[('features',
                                  FeatureUnion(transformer_list=[('one_hot_encode',
                                                                  ColumnTransformer(transformers=[('one_hot_encode',
                                                                                                   OneHotEncoder(sparse=False),
                                                                                                   ['maker',
                                                                                                    'transmission',
                                                                                                    'fuelType'])])),
                                                                 ('robust_encoding',
                                                                  ColumnTransformer(transformers=[('robust_encoding',
                                                                                                   RobustScaler(),
                                                                                                   ['mileage'])])),
                                                                 ('just_passs',
                                                                  ColumnTransformer(transformers=[('passthrough',
                                                                                                   'passthrough',
                                                                                                   ['year',
                                                                                                    'engineSize'])])),
                                                                 ('scale_and_impute',
                                                                  ColumnTransformer(transformers=[('standard_scaling',
                                                                                                   Pipeline(steps=[('impute',
                                                                                                                    SimpleImputer()),
                                                                                                                   ('scale',
                                                                                                                    MinMaxScaler())]),
                                                                                                   ['mpg',
                                                                                                    'tax'])]))]))])),
                ('estimator', LinearRegression())])

In [52]:
train_pred = predicting_pipeline.predict(train)
val_pred = predicting_pipeline.predict(val)

In [53]:
pd.DataFrame({'real':val['price'], 'predicted':val_pred})

,real,predicted
66112,15500,16968.453125
75315,32000,28231.171875
97509,33450,27946.062500
56584,7999,6730.015625
86334,13672,14153.265625
...,...,...
55636,12995,13704.093750
76774,12295,12310.500000
94609,18195,20208.437500
69510,43740,25869.609375


- Evaluación de modelo

In [55]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [56]:
train_mse = mean_absolute_error(train['price'], train_pred)
val_mse = mean_absolute_error(val['price'], val_pred)

print(f"Entrenamiento MSE: {train_mse:2.02f}\n"
      f"Validación MSE:    {val_mse:2.02f}")

Entrenamiento MSE: 2929.39
Validación MSE:    3008.10


In [ ]:
# Evaluación en los datos de prueba

In [57]:
test_pred = predicting_pipeline.predict(test)
test_mse = mean_absolute_error(test['price'], test_pred)

print(f"Prueba MSE: {test_mse:2.02f}")

Prueba MSE: 2928.31


- Guarda el pipeline

In [ ]:
# Joblib nos ayuda a guardar arreglos de forma más eficiente
# dump permite guardar el pipeline

In [58]:
from joblib import dump, load
dump(predicting_pipeline, 'car-prices.model') 

['car-prices.model']